In [ ]:
# use the CPU to train the model.
# ignore it if GPU memory is sufficient.
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
# parameter.

Epochs       = 60
Batch_size   = 32
Channels     = 9                          # representing 9 electron orbitals
Split_ratio  = 0.2
Seed         = 34596                      # random seed to split data
#Seed         = np.random.randint(1, 1e6)
Orbitals     = ['s','py','pz','pz','dxy','dyz','dz2','dxz','dx2']

In [ ]:
import numpy as np
import pickle
import time
import gc

import keras
from keras.preprocessing import sequence
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation, Input, Reshape, BatchNormalization
from keras.layers import (
    Conv1D,
    GlobalAveragePooling1D,
    MaxPooling1D,
    GlobalAveragePooling1D,
    Reshape,
    AveragePooling1D,
    Flatten,
    Concatenate,
)
from keras import backend
from keras.callbacks import TensorBoard, LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# the model for feature extraction
def dos_featurizer(channels):
    # the input data is a 2000*9 matrix
    input_DOS = Input( shape=(2000,channels) )
    # building a feature extraction network
    feature1 = AveragePooling1D(pool_size=  4,strides=4,padding="same")(input_DOS)
    feature2 = AveragePooling1D(pool_size= 25,strides=4,padding="same")(input_DOS)
    feature3 = AveragePooling1D(pool_size=200,strides=4,padding="same")(input_DOS)
    Feature  = Concatenate(axis=-1)([feature1,feature2,feature3])                   # splicing
    Feature  = Conv1D( 50,20,activation="relu", padding="same", strides=2)(Feature) # convolution
    Feature  = BatchNormalization()(Feature)                                        # normalization
    Feature  = Conv1D( 75, 3,activation="relu",padding="same",strides=2)(Feature)   # convolution
    Feature  = AveragePooling1D(pool_size=3,strides=2,padding="same")(Feature)      # pooling
    Feature  = Conv1D(100, 3,activation="relu",padding="same",strides=2)(Feature)   # convolution
    Feature  = AveragePooling1D(pool_size=3,strides=2,padding="same")(Feature)      # pooling
    Feature  = Conv1D(125, 3,activation="relu",padding="same",strides=2)(Feature)   # convolution
    Feature  = AveragePooling1D(pool_size=3,strides=2,padding="same")(Feature)      # pooling
    Feature  = Conv1D(150, 3,activation="relu",padding="same",strides=1)(Feature)   # convolution
    Featurizer_model = Model( input_dos, Feature )                                  # modeling
    return Featurizer_model

# the model for prediction
def create_model(Featurizer, channels):
    # data on 3 layers of topology structure
    input1 = Input( shape=(2000,channels) )
    input2 = Input( shape=(2000,channels) )
    input3 = Input( shape=(2000,channels) )
    # feature extraction
    output1 = Featurizer(input1)
    output2 = Featurizer(input2)
    output3 = Featurizer(input3)
    Output = Concatenate(axis=-1)([output1,output2,output3])
    Output = Flatten()(Output)
    Output = Dropout(0.2)(Output)
    Output = Dense(1000,activation="linear")(Output)
    Output = Dense(2200,activation="relu"  )(Output)
    Output = Dense(2200,activation="relu"  )(Output)
    # modeling
    model = Model( inputs=[input1,input2,input3],outputs=Output )
    return model

# learning function
def decay_schedule(epoch, lr):
    if   epoch== 0: lr = 0.00100
    elif epoch==15: lr = 0.00050
    elif epoch==35: lr = 0.00010
    elif epoch==45: lr = 0.00005
    elif epoch==55: lr = 0.00001
    return lr

In [ ]:
# read data
with open('DataToPredictDOS','rb') as One:
    x_in = pickle.load(One)
    y_in = pickle.load(One)

In [ ]:
###################################################################################
#if you want to predict DOS without training the model by yourself, skip this step#
#                  ,or download data following "Readme".                          #
###################################################################################
# storing predict data
# predict_dos = np.zeros([x_in_unpredict.shape[0],2200,9])

# train
for i in range(9):
    
    gc.collect()
    
    x_train, x_test, y_train, y_test = train_test_split(x_in,
                                                        y_in[:,:,i+1],
                                                        test_size=Split_ratio, 
                                                        random_state=Seed
                                                       )
    shared_conv  = dos_featurizer(Channels)
    lr_scheduler = LearningRateScheduler(decay_schedule, verbose=0)
    tensorboard  = TensorBoard(log_dir="logs/{}".format(time.time()),histogram_freq=1)
    model = create_model(shared_conv, Channels)
    model.compile(loss="logcosh",optimizer=Adam(0.001),metrics=["mean_absolute_error"])
    model.summary()
    model.fit([x_train[:,:,0:9],x_train[:,:,9:18],x_train[:,:,18:27]],
              y_train,
              batch_size=Batch_size,
              epochs=Epochs,
              validation_data=([x_test[:,:,0:9], x_test[:,:,9:18], x_test[:,:,18:27]],y_test),
              callbacks=[tensorboard, lr_scheduler],
             )
        
    train_out = model.predict([x_train[:,:,0:9],x_train[:,:,9:18],x_train[:,:,18:27]])
    train_out = train_out.reshape(train_out.shape)
    test_out  = model.predict([ x_test[:,:,0:9], x_test[:,:,9:18], x_test[:,:,18:27]])
    test_out  =  test_out.reshape( test_out.shape)

    print("train MAE :", mean_absolute_error(y_train, train_out))
    print("train RMSE:",  mean_squared_error(y_train, train_out)**(0.5))
    print("test  MAE :", mean_absolute_error( y_test,  test_out))
    print("test  RMSE:",  mean_squared_error( y_test,  test_out)**(0.5))
    
    print("Saving model...")
    model.save('Model/predict_DOS_'+Orbitals[i]+'.h5')
    
    #predict_dos[:,:,i] = model.predict([x_in_unpredict[:, :, 0:9], x_in_unpredict[:, :, 9:18], x_in_unpredict[:, :, 18:27]])

In [ ]:
################################################################
# Please provide input according to your needs, or use example #
################################################################
# provide input data for prediction.
# according to the introduction of the paper,
# provided DOS data for 9 electron orbitals (s,py,pz,pz,dxy,dyz,dz2,dxz,dx2) of 3 layers 
# has an accuracy of 0.01eV within ±10eV, to make a 2000*27 matrix.

# our example:
List = np.arange(x_in.shape[0])
np.random.shuffle(List)
x_in_unpredict = x_in[List[0:100]]
#y_in_unpredict = y_in[List[0:100]]  # open it if you want compare the data and prediction

predict_dos = np.zeros([x_in_unpredict.shape[0],2200,9])
for i in range(9):
    Model = load_model('Model/predict_DOS_'+Orbitals[i]+'.h5')
    predict_dos[:,:,i] = Model.predict([x_in_unpredict[:, :, 0:9],x_in_unpredict[:, :, 9:18],x_in_unpredict[:, :, 18:27]])
    gc.collect()